# AWS S3 Public Access Block Removed

Detect when the S3 Public Access Block configuration is removed from an S3 bucket or an AWS account. This could indicate an attempt to impair defenses by making resources open to unauthorized users.

## Detection Query
Execute the following query to find alerts related to this rule.

In [ ]:
%ingest.source_type="aws:cloudtrail"
eventSource:(
"s3.amazonaws.com"
"s3control.amazonaws.com"
)
eventName:(
DeletePublicAccessBlock
PutPublicAccessBlock
DeleteBucketPublicAccessBlock
PutBucketPublicAccessBlock
DeleteAccountPublicAccessBlock
PutAccountPublicAccessBlock
)
not errorCode:*
(
eventName:(
DeletePublicAccessBlock
DeleteBucketPublicAccessBlock
DeleteAccountPublicAccessBlock
)
or
(
eventName:(
PutPublicAccessBlock
PutBucketPublicAccessBlock
PutAccountPublicAccessBlock
)
(
requestParameters.PublicAccessBlockConfiguration.BlockPublicAcls:"false"
or requestParameters.PublicAccessBlockConfiguration.IgnorePublicAcls:"false"
or requestParameters.PublicAccessBlockConfiguration.BlockPublicPolicy:"false"
or requestParameters.PublicAccessBlockConfiguration.RestrictPublicBuckets:"false"
)
)
)
| groupbycount(
recipientAccountId,
userIdentity.arn,
awsRegion
)
| where @q.count > 0

## Investigation Steps
1. **Identify the Principal**: Who performed the action?
2. **Review Context**: What else did this principal do?
3. **Check Permissions**: Did the principal have legitimate reasons for this action?
4. **Verify Resource**: Is the resource critical or sensitive?

In [ ]:
%ingest.source_type:aws:cloudtrail
userIdentity.arn: <PRINCIPAL_ARN>
| count(eventName) by eventName